In [1]:
from file_storage import FileStorage
import keras
from bs4 import BeautifulSoup
import re
import transliterate
import tqdm
import unicodedata
from collections import Counter
import numpy as np
import pandas as pd

Using TensorFlow backend.


In [87]:
TRANSLITERATE_DICT = {
    'а': 'a',
    'б': 'b',
    'в': 'v',
    'г': 'g',
    'д': 'd',
    'е': 'e',
    'ж': 'zh',
    'з': 'z',
    'и': 'i',
    'к': 'k',
    'л': 'l',
    'м': 'm',
    'н': 'n',
    'о': 'o',
    'п': 'p',
    'р': 'r',
    'с': 's',
    'т': 't',
    'ф': 'f',
    'х': 'h',
    'ц': 'ts',
    'ч': 'ch',
    'ш': 'sh',
    'щ': 'sch',
    'ъ': "'",
    'ы': 'y',
    'ь': "'",
    'э': 'e',
    'ю': 'ju',
    'я': 'ya',
    'π': 'pi',
    'ı': 'i',
    'ə': 'e',
    'ل': 'j',
    'ƒ': 'f',
    'ﬁ': 'fi',
    '\xad': '-',
    'µ': 'mu',
    '\u200b': ' ',
    'ː': ':',
    '—': '-',
    '−': '-',
    '–': '-',
    '”': '"',
    '“': '"',
    '«': '"',
    '»': '"',
    'у': 'y',
    '’': '"',
    '‘': '"',
    '`': "'",
    '„': '"',
    '·': ',',
    '•': ',',
    '…': ' ',
    # https://www.redhat.com/archives/fedora-extras-commits/2007-June/msg03617.html
    "\u0621": "'", # hamza-on-the-line
    "\u0622": "|", # madda
    "\u0623": ">", # hamza-on-'alif
    "\u0624": "&", # hamza-on-waaw
    "\u0625": "<", # hamza-under-'alif
    "\u0626": "}", # hamza-on-yaa'
    "\u0627": "A", # bare 'alif
    "\u0628": "b", # baa'
    "\u0629": "p", # taa' marbuuTa
    "\u062A": "t", # taa'
    "\u062B": "v", # thaa'
    "\u062C": "j", # jiim
    "\u062D": "H", # Haa'
    "\u062E": "x", # khaa'
    "\u062F": "d", # daal
    "\u0630": "*", # dhaal
    "\u0631": "r", # raa'
    "\u0632": "z", # zaay
    "\u0633": "s", # siin
    "\u0634": "$", # shiin
    "\u0635": "S", # Saad
    "\u0636": "D", # Daad
    "\u0637": "T", # Taa'
    "\u0638": "Z", # Zaa' (DHaa')
    "\u0639": "E", # cayn
    "\u063A": "g", # ghayn
    "\u0640": "_", # taTwiil
    "\u0641": "f", # faa'
    "\u0642": "q", # qaaf
    "\u0643": "k", # kaaf
    "\u0644": "l", # laam
    "\u0645": "m", # miim
    "\u0646": "n", # nuun
    "\u0647": "h", # haa'
    "\u0648": "w", # waaw
    "\u0649": "Y", # 'alif maqSuura
    "\u064A": "y", # yaa'
    "\u064B": "F", # fatHatayn
    "\u064C": "N", # Dammatayn
    "\u064D": "K", # kasratayn
    "\u064E": "a", # fatHa
    "\u064F": "u", # Damma
    "\u0650": "i", # kasra
    "\u0651": "~", # shaddah
    "\u0652": "o", # sukuun
    "\u0670": "`", # dagger 'alif
    "\u0671": "{", # waSlaﬁ
}

In [94]:
from urllib.request import urlopen
import urllib


def download_from_the_internet(url):
    try:
        return urlopen(url).read().decode('utf-8')
    except KeyboardInterrupt:
        raise
    except urllib.error.HTTPError as e:
        code = e.code
        if code != 404:
            print(e)
        return code
    except Exception as e:
        print(e)

In [90]:
# https://stackoverflow.com/questions/34753821/remove-diacritics-from-string-for-search-function

def shave_marks(txt):
    """This method removes all diacritic marks from the given string"""
    norm_txt = unicodedata.normalize('NFD', txt)
    shaved = ''.join(c for c in norm_txt if not unicodedata.combining(c))
    return unicodedata.normalize('NFC', shaved)


def is_english_letters(string):
    return re.search(r'[^a-zA-Z0-9°_©®™;§,№!#@.×:+=()/£¥€$|<>~{}\\\[\]%&*^?"\'-]', string) is None


def try_transliterate(query):
    query = unicodedata.normalize('NFC', shave_marks(query).lower())
    try:
        return transliterate.translit(query, reversed=True)
    except transliterate.exceptions.LanguageDetectionError as query_error:
        transliteration = []
        for word in query.split():
            if is_english_letters(word):
                transliteration.append(word)
            else:
                try:
                    transliteration.append(transliterate.translit(word, reversed=True))
                except transliterate.exceptions.LanguageDetectionError as e:
                    new_word = []
                    for ch in word:
                        translited_ch = TRANSLITERATE_DICT.get(ch, ch)
                        new_word.append(translited_ch)
                    transliteration.append(''.join(new_word))
        return ' '.join(transliteration) 

In [91]:
with open('req_ans_learn.tsv', encoding='utf-8-sig') as train_file, open('transliterated_learn.tsv', 'w') as transliterated_learn_file:
    for line in tqdm.tqdm(train_file):
        query, url_end = line.strip().split('\t')
        transliteration = try_transliterate(query)
        transliterated_learn_file.write(transliteration + '\t' + url_end + '\n')

500000it [00:55, 9059.98it/s] 


In [2]:
file_storage = FileStorage('../filtered_storage')

In [3]:
len(file_storage)

151120

In [7]:
# https://www.quora.com/How-can-I-extract-only-text-data-from-HTML-pages

def get_text(html):
    soup = BeautifulSoup(html)
    data = soup.findAll(text=True)

    def informative(element):
        if element.parent.name in ['style', 'script', '[document]', 'head', 'title']:
            return False
        elif re.match('<!--.*-->', str(element.encode('utf-8'))):
            return False
        elif len(element) < 20:
            return False
        return True

    return [line for line in data if informative(line)]

In [8]:
BEGINNING = 'https://simple.wikipedia.org'

In [9]:
get_text(file_storage.read(BEGINNING + '/wiki/Germany'))

['From Wikipedia, the free encyclopedia',
 'This page is about the modern country named ',
 '.  For other meanings of Germany, see ',
 'Federal Republic of Germany',
 'Bundesrepublik Deutschland',
 'Einigkeit und Recht und Freiheit',
 '"Unity and Justice and Freedom"',
 'and national language',
 '7001790000000000000♠',
 '7001110000000000000♠',
 '7000700000000000000♠',
 '7000230000000099999♠',
 '7000350000000000000♠',
 '6999800000000000000♠',
 '6999500000000000000♠',
 '7000170000000000000♠',
 '0.5% Other religions',
 'Frank-Walter Steinmeier',
 'President of the Bundestag',
 'President of the Bundesrat',
 'President of the Federal',
 ' Constitutional Court',
 '•\xa030 June 2016 estimate',
 'Federal Republic of Germany',
 'Bundesrepublik Deutschland',
 ". The country's full name is sometimes shortened to the ",
 'To the north of Germany are the ',
 ', and the kingdom of ',
 '. To the east of Germany are the countries of ',
 '. To the south of Germany are the countries of ',
 '. To the we

In [88]:
def get_words(string):
    return [word for word in str(string).lower().split(',.!?;:() \t\n') if word]


def get_ngrams(n, word_list):
    if n == 1:
        return ''.join(word_list)
    else:
        return [
            word[i:i + n]
            for word in word_list
            for i in range(len(word) - n + 1)
        ]


def get_n_gram_counter(n, collection):
    counter = Counter()
    for element in collection:
        counter.update(get_ngrams(n, get_words(element)))
    return counter

In [89]:
class Storage:
    def __init__(self, elements):
        self._ind_to_elem = elements
        self._elem_to_ind = {elem: ind for ind, elem in enumerate(elements)}
        
    def get_elem(self, ind):
        return self._ind_to_elem[ind]
    
    def get_ind(self, elem, default=None):
        return self._elem_to_ind.get(elem, default)
    
    def __len__(self):
        return len(self._ind_to_elem)
    
    @classmethod
    def from_counter(cls, counter, n_most_common):
        return Storage([elem for elem, _ in counter.most_common(n_most_common)])
    

class Encoder:
    def encode(self, example):
        raise NotImplemented()

    def encode_with_padding(self, examples):
        codes = [self.encode(example) for example in examples]
        max_len = max(map(len, codes))
        return np.array([
            np.concatenate([code, np.zeros(len(code) - max_len)])
        ], dtype=np.int32)


class BagOfNgramsEncoder(Encoder):
    def __init__(self, collection, ngram_number_array):
        self._ngram_storages = [
            Storage.from_counter(get_n_gram_counter(n + 1, collection), ngram_number)
            for n, ngram_number in enumerate(ngram_number_array)
        ]
        self._code_size = sum(map(len, self._ngram_storages)) + 2
    
    @property
    def n(self):
        return len(self._ngram_storages)
    
    def _encode_word(self, word):
        for i in range(len(word) - self.n + 1):
            ind = 1
            for ngram_len in reversed(range(1, self.n + 1)):
                ngram = word[i:i + ngram_len]
                ngram_ind = self._ngram_storages[ngram_len - 1].get_ind(ngram)
                if ngram_ind is None:
                    ind += len(self._ngram_storages[ngram_len - 1])
                else:
                    ind += ngram_ind
                    return ind
            return ind

    def encode(self, string):
        return sum((self._encode_word(word) for word in get_words(string)), [])
        
    @property
    def code_size(self):
        return self._code_size

In [113]:
class DocEncoder(Encoder):    
    def __init__(self, storage, collection, n_words, n_trigrams):
        trigram_counter = Counter()
        word_counter = Counter()
        
        for example in tqdm.tqdm(collection):
            html = storage.read(BEGINNING + example)
            if html is None:
                print(example)
                html = download_from_the_internet(BEGINNING + example)
                if isinstance(html, int) or html is None:
                    continue
                file_storage.write(BEGINNING + example, html)
            text = get_text(html)
            words, trigrams = self._prepare_example(text)
            word_counter.update(words)
            trigram_counter.update(trigrams)

        self._word_storage = Storage.from_counter(word_counter, n_words)
        self._trigram_storage = Storage.from_counter(trigram_counter, n_trigrams)    
    
    @staticmethod
    def _prepare_example(example):
        words = []
        trigrams = []
        for example_part in example:
            curr_words = get_words(example_part)
            words += curr_words
            trigrams += get_ngrams(3, curr_words)
        return words, trigrams
    
    def _encode_word(self, word):
        word_code = self._word_storage.get_ind(word)
        if word_code is None:
            return [
                self._trigram_storage.get_ind(trigram, self._unk_ind)
                for trigram in get_ngrams(3, [word])
            ]
        else:
            return [word_code]
    
    def encode(self, example):
        return sum(
            (
                self._encode_word(word)
                for word in get_words(example_part)
                for example_part in example
            ), []
        )
        
    @property
    def _unk_ind(self):
        return self.code_size - 1
                   
    @property
    def _pad_ind(self):
        return 0
                                                  
    @property
    def code_size(self):
        return len(self._word_storage) + len(self._trigram_storage) + 2

In [74]:
query_encoder = BagOfNgramsEncoder(queries_train, [500, 2500, 12000])

In [114]:
doc_encoder = DocEncoder(file_storage, docs_train, 15000, 10000)


  0%|          | 818/463852 [00:20<2:52:46, 44.66it/s]


  0%|          | 1614/463852 [00:41<4:58:31, 25.81it/s]


  0%|          | 1806/463852 [00:46<4:17:50, 29.87it/s]

/wiki/Dil_Hi_Toh_Hai



  0%|          | 1824/463852 [00:48<7:51:39, 16.33it/s]

...


 10%|▉         | 46227/463852 [43:35<4:47:50, 24.18it/s]

/wiki/Francysk_Skaryna



 10%|▉         | 46273/463852 [43:37<3:13:20, 36.00it/s]

KeyboardInterrupt: 


 10%|▉         | 46273/463852 [43:52<3:13:20, 36.00it/s]

In [31]:
def get_tiled(x_positive, y_positive, y_negative, batch_size, negative_cnt):
    return [
        np.tile(x_positive, (1 + negative_cnt, 1)),
        np.concatenate([y_positive, np.repeat(y_negative, batch_size, axis=0)], axis=0),
    ]


def batch_generator(queries, doc_names, doc_storage, doc_encoder, query_encoder, batch_size, negative_cnt=2):
    while True:
        indexes = np.random.choice(np.arange(len(queries)), batch_size + negative_cnt, replace=False)
        batch_indexes = indexes[:batch_size]
        query_codes = query_encoder.encode_with_padding(queries[indexes])
        positive_query_codes = query_codes[:batch_size]
        negative_query_codes = query_codes[batch_size:]
        doc_codes = doc_encoder.encode_with_padding([
            get_text(doc_storage.read(BEGINNING + name))
            for name in doc_names[batch_indexes]
        ])
        yield (
            get_tiled(doc_codes, positive_query_codes, negative_query_codes, batch_size, negative_cnt),
            np.concatenate([np.ones(batch_size), np.zeros(batch_size * negative_cnt)]),
        )

In [20]:
def read_data(path):
    data = pd.read_csv(path, sep='\t', header=None).values
    return data[:, 0], data[:, 1]


def dump(path, *arrays):
    data_frame = pd.DataFrame(data=np.transpose(np.array(arrays)))
    data_frame.to_csv(path, sep='\t', header=False, index=False)

In [83]:
from sklearn.model_selection import train_test_split


queries, docs = read_data('transliterated_learn.tsv')
queries_train, queries_val, docs_train, docs_val = train_test_split(queries, docs, test_size=0.05, random_state=1)

In [84]:
dump('train.tsv', queries_train, docs_train)
dump('val.tsv', queries_val, docs_val)

In [22]:
def get_dense(units, activation='relu'):
    return keras.layers.Dense(
        units, activation=activation, kernel_regularizer=keras.regularizers.l2(0.01),
    )

In [23]:
doc = keras.layers.Input(shape=(None,), dtype='int32')
query = keras.layers.Input(shape=(None,), dtype='int32')

def get_embed(embed_layers, data):
    for layer in embed_layers:
        data = layer(data)
    return data


doc_embed_layers = [
    keras.layers.Embedding(encoder.code_size, 256),
    keras.layers.LSTM(256),
    keras.layers.Dropout(0.2),
    get_dense(256),
    keras.layers.Dropout(0.2),
    get_dense(128, activation=None),
]

query_embed_layers = [
    keras.layers.Embedding(encoder.code_size, 128),
    keras.layers.LSTM(128),
    keras.layers.Dropout(0.2),
    get_dense(128),
    keras.layers.Dropout(0.2),
    get_dense(128, activation=None),
]

doc_embed = get_embed(doc_embed_layers, doc)
query_embed = get_embed(query_embed_layers, query)

similarity = keras.layers.Dot(axes=1, normalize=True)([doc_embed, query_embed])

model = keras.models.Model(
    inputs=[doc, query],
    outputs=similarity,
)

model.compile(keras.optimizers.Adam(), loss='binary_crossentropy')

NameError: name 'encoder' is not defined

In [ ]:
batch_size = 64
negative_cnt = 3

training_history = model.fit_generator(
    generator=batch_generator(
        queries_train, docs_train, file_storage, doc_encoder, query_encoder, batch_size, negative_cnt
    ),
    epochs=1,
    steps_per_epoch=2000,
    callbacks=[
        keras.callbacks.TensorBoard(batch_size=batch_size),
        keras.callbacks.ModelCheckpoint('big_classifier.bin', monitor='val_loss', save_best_only=True),
        keras.callbacks.ReduceLROnPlateau(factor=0.2, patience=5, min_delta=0.0001),
    ],
    validation_data=batch_generator(
        queries_val, docs_val, file_storage, doc_encoder, query_encoder, batch_size, negative_cnt
    ),
    validation_steps=200,
)